# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

# Configurations

In [ ]:
import findspark
findspark.init('/home/charles/spark-3.1.1-bin-hadoop2.7')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacker').getOrCreate()

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled',True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows',5)

# Imports

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import MinMaxScaler,VectorAssembler

import pyspark.sql.functions as F

# Data

In [ ]:
data = spark.read.csv('hack_data.csv',inferSchema=True,header=True)

In [ ]:
data

Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
8.0,391.09,1,2.96,7.0,Slovenia,72.37
20.0,720.99,0,3.04,9.0,British Virgin Is...,69.08
31.0,356.32,1,3.71,8.0,Tokelau,70.58
2.0,228.08,1,2.48,8.0,Bolivia,70.8
20.0,408.5,0,3.57,8.0,Iraq,71.28


In [ ]:
data = data.drop('Location')

In [ ]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



# Spark Formatting

In [ ]:
assembler = VectorAssembler(inputCols=data.columns,outputCol='features')

In [ ]:
fmtd_data = assembler.transform(data)

# Scaling

In [ ]:
scaler = MinMaxScaler(min=0, max=1, inputCol='features', outputCol='scaledFeatures')

In [ ]:
fmtd_scld_data = scaler.fit(fmtd_data).transform(fmtd_data)

In [ ]:
fmtd_scld_data

Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,WPM_Typing_Speed,features,scaledFeatures
8.0,391.09,1,2.96,7.0,72.37,"[8.0,391.09,1.0,2...",[0.11864406779661...
20.0,720.99,0,3.04,9.0,69.08,"[20.0,720.99,0.0,...",[0.32203389830508...
31.0,356.32,1,3.71,8.0,70.58,"[31.0,356.32,1.0,...",[0.50847457627118...
2.0,228.08,1,2.48,8.0,70.8,"[2.0,228.08,1.0,2...",[0.01694915254237...
20.0,408.5,0,3.57,8.0,71.28,"[20.0,408.5,0.0,3...",[0.32203389830508...


# Clustering

In [ ]:
# Two Clusters (Two Hackers)

In [ ]:
kmeans2 = KMeans(k=2, featuresCol='scaledFeatures')

In [ ]:
k2_model = kmeans2.fit(fmtd_scld_data)

In [ ]:
k2_model.summary.clusterSizes

[167, 167]

In [ ]:
# Three Clusters

In [ ]:
kmeans3 = kmeans2.setK(3)

In [ ]:
k3_model = kmeans3.fit(fmtd_scld_data)

In [ ]:
k3_model.summary.clusterSizes

[167, 84, 83]